In [ ]:
## Sentiment, sentiment_data.csv in shared file is utilized
## 576 days without sentiment score between 2009-07-27 & 2020-06-11
## sentiment_df only has aggregated score daily, sentiment_percentages has daily headline percentage
## negative:0, neutral:1, positive:2

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

sent_path =  "/content/drive/My Drive/Main model working folder/sentiment_data.csv"

sentiment = pd.read_csv(sent_path, parse_dates=["Date"])
sentiment['Date'] = pd.to_datetime(sentiment['Date'])
sentiment['Date'] = sentiment['Date'].dt.tz_localize(None)
sentiment['Date'] = pd.to_datetime(sentiment['Date']).dt.tz_localize(None).dt.normalize()
sentiment_df = sentiment[['Date','sentiment_encoded']]

sentiment_counts = sentiment.groupby(['Date', 'sentiment_encoded']).size().unstack(fill_value=0)
sentiment_counts['total'] = sentiment_counts.sum(axis=1)
sentiment_percentages = sentiment_counts.div(sentiment_counts['total'], axis=0)
sentiment_percentages = sentiment_percentages.rename(columns={
    'positive': 'positive_pct',
    'negative': 'negative_pct',
    'neutral': 'neutral_pct'
})

sentiment_df = sentiment.groupby("Date")[["sentiment_encoded"]].mean().reset_index()
sentiment_df = sentiment_df.sort_values("Date")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


sentiment_encoded,0,1,2,total
Date,,,,
2009-07-27,0,0,1,1
2009-08-07,0,0,1,1
2009-08-10,3,0,3,6
2009-08-12,0,0,1,1
2009-08-13,1,0,1,2


In [ ]:
sentiment_df["Date"].min(), sentiment_df["Date"].max()

(Timestamp('2009-07-27 00:00:00'), Timestamp('2020-06-11 00:00:00'))

In [ ]:
## after merging with other data, can fill sentiment score for missing days with previous reported sentiment score
DF['sentiment_encoded'] = DF['sentiment_encode'].fillna(method='ffill')


(3396, 2)

In [ ]:
# macroeconomic trends (CPI, etc.)
#%pip install fredapi

from fredapi import Fred
import pandas as pd
fred = Fred(api_key='dab4815a04eb85f8ef19813a52a64180')

# Retrieve data
cpi = fred.get_series('CPIAUCSL')               # Consumer Price Index
unemployment = fred.get_series('UNEMPLOY')      # Unemployment Rate
interest_rate = fred.get_series('FEDFUNDS')     # Federal Funds Rate
oil_price = fred.get_series('DCOILWTICO')       # West Texas Intermediate Crude
egg_price = fred.get_series('APU0000708111')    # Average price: eggs
unemployment_rate = fred.get_series('UNRATE')   # U.S. Unemployment Rate
gdp_growth = fred.get_series('A191RL1Q225SBEA') # Real GDP (percent change)
treasury_yield_10y = fred.get_series('DGS10')   # Treasury Yield (10-Year)
gold_price = fred.get_series('IR14270')         # Gold Price (Daily from IMF)
usd_index = fred.get_series('DTWEXBGS')         # U.S. Dollar Index (Nominal Broad)



# Combine into a DataFrame
macroindicators_df = pd.DataFrame({
    'CPI': cpi,
    'Unemployment Rate': unemployment,
    'Federal Funds Rate': interest_rate,
    'Oil Price': oil_price,
    'Egg Price': egg_price,
    'Unemployment Rate': unemployment_rate,
    'Real GDP Growth': gdp_growth,
    'Treasury Yield (10-Year)': treasury_yield_10y,
    'Gold Price': gold_price,
    'U.S. Dollar Index': usd_index

})

# Display the first few rows
macroindicators_df.reset_index(inplace=True)
macroindicators_df.rename(columns={'index': 'Date'}, inplace=True)
macroindicators_df['Date'] = pd.to_datetime(macroindicators_df['Date'])
print(macroindicators_df.head())


        Date    CPI  Unemployment Rate  Federal Funds Rate  Oil Price  \
0 1947-01-01  21.48                NaN                 NaN        NaN   
1 1947-02-01  21.62                NaN                 NaN        NaN   
2 1947-03-01  22.00                NaN                 NaN        NaN   
3 1947-04-01  22.00                NaN                 NaN        NaN   
4 1947-05-01  21.95                NaN                 NaN        NaN   

   Egg Price  Real GDP Growth  Treasury Yield (10-Year)  Gold Price  \
0        NaN              NaN                       NaN         NaN   
1        NaN              NaN                       NaN         NaN   
2        NaN              NaN                       NaN         NaN   
3        NaN             -1.0                       NaN         NaN   
4        NaN              NaN                       NaN         NaN   

   U.S. Dollar Index  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                Na

In [ ]:
# Enhanced Sentiment Model with Random Forest and XGBoost
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier, XGBRegressor
import matplotlib.pyplot as plt
drive.mount('/content/drive')
sent_path = "/content/drive/My Drive/DAT490 Working Folder/Main model working folder/sentiment_data.csv"
sp500_path = "/content/drive/My Drive/DAT490 Working Folder/Main model working folder/GSPCIndicators2.csv"


# Load and process sentiment data
sentiment = pd.read_csv(sent_path, parse_dates=["Date"])
sentiment['Date'] = pd.to_datetime(sentiment['Date']).dt.normalize()
sentiment['Date'] = pd.to_datetime(sentiment['Date']).dt.tz_localize(None).dt.normalize()
sentiment_df = sentiment.groupby("Date")["sentiment_encoded"].mean().reset_index()

# Load and process S&P 500 indicators
gspc = pd.read_csv(sp500_path, parse_dates=["Date"])
features_to_use = [col for col in gspc.columns if col != "Date"]
gspc_df = gspc[['Date'] + features_to_use]

# Merge and target creation
merged_df = pd.merge(sentiment_df, gspc_df, on="Date", how="left")
merged_df = merged_df.sort_values("Date")
merged_df["target_return"] = merged_df["GSPC.Close"].pct_change().shift(-1)
merged_df["target_direction"] = (merged_df["target_return"] > 0).astype(int)

# Lagged sentiment and return
merged_df['sentiment_lag1'] = merged_df['sentiment_encoded'].shift(1)
merged_df['return_lag1'] = merged_df['target_return'].shift(1)

# Drop rows with missing data
model_df = merged_df.dropna()

# Feature selection
exclude_cols = ['Date', 'target_return', 'target_direction']
target_cols = ['target_direction', 'target_return']
features = model_df.drop(columns=exclude_cols + ['GSPC.Adjusted'])
scaler = StandardScaler()
X = scaler.fit_transform(features)
y_class = model_df['target_direction']
y_reg = model_df['target_return']

# Time series split
tscv = TimeSeriesSplit(n_splits=5)

# Hyperparameter tuning for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Random Forest Classifier
clf_rf = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_clf_search = RandomizedSearchCV(clf_rf, param_grid_rf, cv=tscv, n_iter=10, scoring='accuracy', random_state=42)
rf_clf_search.fit(X, y_class)
print("\nBest RF Classifier Params:", rf_clf_search.best_params_)
y_pred_rf_class = rf_clf_search.predict(X)
print("\nRF Classification Report:\n", classification_report(y_class, y_pred_rf_class))

# Random Forest Regressor
reg_rf = RandomForestRegressor(random_state=42)
rf_reg_search = RandomizedSearchCV(reg_rf, param_grid_rf, cv=tscv, n_iter=10, scoring='neg_mean_squared_error', random_state=42)
rf_reg_search.fit(X, y_reg)
print("\nBest RF Regressor Params:", rf_reg_search.best_params_)
y_pred_rf_reg = rf_reg_search.predict(X)
print("\nRF Regression RMSE:", np.sqrt(mean_squared_error(y_reg, y_pred_rf_reg)))
print("R^2 Score:", r2_score(y_reg, y_pred_rf_reg))

# XGBoost Classifier
clf_xgb = XGBClassifier(random_state=42, eval_metric='logloss')
clf_xgb.fit(X, y_class)
y_pred_xgb_class = clf_xgb.predict(X)
print("\nXGBoost Classification Report:\n", classification_report(y_class, y_pred_xgb_class))

# XGBoost Regressor
reg_xgb = XGBRegressor(random_state=42)
reg_xgb.fit(X, y_reg)
y_pred_xgb_reg = reg_xgb.predict(X)
print("\nXGBoost Regression RMSE:", np.sqrt(mean_squared_error(y_reg, y_pred_xgb_reg)))
print("R^2 Score:", r2_score(y_reg, y_pred_xgb_reg))

# Plot regression results
plt.figure(figsize=(12, 6))
plt.plot(y_reg.values, label="Actual", linewidth=2)
plt.plot(y_pred_rf_reg, label="RF Predicted", linestyle='--')
plt.plot(y_pred_xgb_reg, label="XGB Predicted", linestyle='dotted')
plt.legend()
plt.title("Regression: Actual vs Predicted Returns")
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot confusion matrix for Random Forest
cm = confusion_matrix(y_class, y_pred_rf_class)
plt.figure(figsize=(6, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("RF Classification Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ["Down/Flat", "Up"])
plt.yticks(tick_marks, ["Down/Flat", "Up"])
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()
